## Walnut Creek TDM - OD Summary by Trip Purpose

This script takes in Streetlight O-D data, summarize the number of OD trips by Day Type, Time of Day, and Trip Purpose.
It outputs heatmaps of OD zones for trips to and from Walnut Creek.

Python 2.7 with Pandas 0.23.4

Input data: 'OD_TP.csv'


1) Day Type: '0: Average Day (M-Su)', '1: Average Weekday (Tu-Th)', '2: Average Weekend Day (Sa-Sa)'

2) Day Part: '00: All Day (12am-12am)', AM Peak - '08: 7am (7am-8am)', '09: 8am (8am-9am)', PM-Peak - '17: 4pm (4pm-5pm)', '18: 5pm (5pm-6pm)'

In [98]:
import os
print os.getcwd(); # Prints the working directory
# set working directory
# os.chdir(default_path)

%matplotlib inline
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
sns.set()

C:\Users\mli\Documents\Scripts\Walnut Creek TDM


In [99]:
# read in O-D Trip Purpose data
odtp_df = pd.read_csv('OD_TP.csv')

In [100]:
#od_df.head() # read first 5 rows of data

In [101]:
#odtp_df['Type of Travel'].unique()

In [102]:
list(odtp_df) # get a list of attribute names

['Type of Travel',
 'Origin Zone ID',
 'Origin Zone Name',
 'Origin Zone Is Pass-Through',
 'Origin Zone Direction (degrees)',
 'Origin Zone is Bi-Direction',
 'Destination Zone ID',
 'Destination Zone Name',
 'Destination Zone Is Pass-Through',
 'Destination Zone Direction (degrees)',
 'Destination Zone is Bi-Direction',
 'Day Type',
 'Day Part',
 'O-D Traffic (StL Index)',
 'Purpose HBW (percent)',
 'Purpose HBO (percent)',
 'Purpose NHB (percent)']

In [103]:
# calculate number of HBW, HBO, NHB trips
odtp_df['HBW Trips'] = odtp_df.apply(lambda x: round(x['O-D Traffic (StL Index)'] * x['Purpose HBW (percent)']), axis=1)
odtp_df['HBO Trips'] = odtp_df.apply(lambda x: round(x['O-D Traffic (StL Index)'] * x['Purpose HBO (percent)']), axis=1)
odtp_df['NHB Trips'] = odtp_df.apply(lambda x: round(x['O-D Traffic (StL Index)'] * x['Purpose NHB (percent)']), axis=1)

# select columns we need
df = odtp_df[['Origin Zone ID','Origin Zone Name','Destination Zone ID', 'Destination Zone Name', 'Origin Zone Is Pass-Through','Destination Zone Is Pass-Through', 'Day Type', 'Day Part','O-D Traffic (StL Index)', 'Purpose HBW (percent)', 'Purpose HBO (percent)', 'Purpose NHB (percent)', 'HBW Trips', 'HBO Trips', 'NHB Trips']]

In [104]:
#df['Day Type'].unique()

In [105]:
#df['Day Part'].unique()

In [106]:
# read in lookup table
zone = pd.read_csv('zone_crosswalk.csv')

# rename columns
origin_zone = zone.rename(index=str, columns={"ZoneID": "Origin Zone ID","Street Light Zone":"Origin Street Light Zone", "Location": "Origin Location", "Group": "Origin Group", "Employment": "Origin Employment"})
destination_zone = zone.rename(index=str, columns={"ZoneID": "Destination Zone ID","Street Light Zone":"Destination Street Light Zone", "Location": "Destination Location", "Group": "Destination Group", "Employment": "Destination Employment"})

In [107]:
#origin_zone

In [108]:
#destination_zone

In [109]:
# join zone lookup table to O-D dataframe
df_o = pd.merge(df, origin_zone, on='Origin Zone ID', validate="many_to_one")
df_od = pd.merge(df_o, destination_zone, on="Destination Zone ID", validate="many_to_one" )

#df_od.head()

In [110]:
# filter out pass-through trips
df_wc= df_od[((df_od['Origin Group'] == 'Walnut Creek') | (df_od['Destination Group'] == 'Walnut Creek')) & (df_od['Origin Zone Is Pass-Through'] == 'no') & (df_od['Destination Zone Is Pass-Through'] == 'no')]
od_wc=df_wc.drop(columns=['Origin Zone Is Pass-Through', 'Destination Zone Is Pass-Through'])
#od_wc.head()

In [111]:
# filter by day part
od_wc_all = od_wc[(od_wc['Day Part'] =='00: All Day (12am-12am)')]
od_wc_am = od_wc[((od_wc['Day Part'] =='08: 7am (7am-8am)')|(od_wc['Day Part'] =='09: 8am (8am-9am)'))]
od_wc_pm = od_wc[((od_wc['Day Part'] =='17: 4pm (4pm-5pm)')|(od_wc['Day Part'] =='18: 5pm (5pm-6pm)'))]

In [112]:
# set group arrays
Day_Type = ['0: Average Day (M-Su)', '1: Average Weekday (Tu-Th)', '2: Average Weekend Day (Sa-Sa)']
Trip_Purpose = ['HBW Trips', 'HBO Trips', 'NHB Trips']
OD = [' Zone Name', ' Location', ' Group']

In [113]:
# loop through group and plot
# daily plot
for x in Day_Type:
    for y in Trip_Purpose:
        for z in OD:
            od_wc_all_dt = od_wc_all[(od_wc_all['Day Type'] == x)]
            # pivot Table
            matrix = od_wc_all_dt.pivot_table(values= y, columns= 'Destination' + z, index='Origin' + z, aggfunc=np.sum)
            # seaborn heatmap documentation
            # https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.heatmap.html
            # https://seaborn.pydata.org/generated/seaborn.heatmap.html
            # color ramps https://matplotlib.org/examples/color/colormaps_reference.html
            # cmap choices: http://matplotlib.org/users/colormaps.html
            plt.figure(figsize=(9,9), facecolor='#FFFFFF',edgecolor='#FFFFFF', clear=True)
            plt.xlabel('Origin' + z, size = 15)
            plt.ylabel('Destination' + z, size = 15)
            plt.title('Daily' + ' ' + x[3:] + ' ' + y, size = 15)
            sns.heatmap(matrix, fmt=".1f", linewidths=.5, square = True, cmap = 'Oranges', annot = None, xticklabels=True, yticklabels=True);
            plt.savefig('Daily'+ x[3:] + y + z + '.pdf', dpi=300, bbox_inches='tight', papertype=None)
            plt.close()

In [114]:
# AM plot
for x in Day_Type:
    for y in Trip_Purpose:
        for z in OD:
            od_wc_am_dt = od_wc_am[(od_wc_am['Day Type'] == x)]
            # pivot Table
            matrix = od_wc_am_dt.pivot_table(values= y, columns= 'Destination' + z, index='Origin' + z, aggfunc=np.sum)
            plt.figure(figsize=(9,9), facecolor='#FFFFFF',edgecolor='#FFFFFF', clear=True)
            plt.xlabel('Origin' + z, size = 15)
            plt.ylabel('Destination' + z, size = 15)
            plt.title('AM' + ' ' + x[3:] + ' ' + y, size = 15)
            sns.heatmap(matrix, fmt=".1f", linewidths=.5, square = True, cmap = 'Reds', annot = None, xticklabels=True, yticklabels=True);
            plt.savefig('AM'+ x[3:] + y + z + '.pdf', dpi=300, bbox_inches='tight', papertype=None)
            plt.close()


In [115]:
# PM plot
for x in Day_Type:
    for y in Trip_Purpose:
        for z in OD:
            od_wc_pm_dt = od_wc_pm[(od_wc_pm['Day Type'] == x)]
            # pivot Table
            matrix = od_wc_pm_dt.pivot_table(values= y, columns= 'Destination' + z, index='Origin' + z, aggfunc=np.sum)
            plt.figure(figsize=(9,9), facecolor='#FFFFFF',edgecolor='#FFFFFF', clear=True)
            plt.xlabel('Origin' + z, size = 15)
            plt.ylabel('Destination' + z, size = 15)
            plt.title('PM' + ' ' + x[3:] + ' ' + y, size = 15)
            sns.heatmap(matrix, fmt=".1f", linewidths=.5, square = True, cmap = 'Blues', annot = None, xticklabels=True, yticklabels=True);
            plt.savefig('PM'+ x[3:] + y + z + '.pdf', dpi=300, bbox_inches='tight', papertype=None)
            plt.close()